In [ ]:
!pip install evaluate


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 7.5 MB/s eta 0:00:00


In [ ]:
!pip install rouge_score

  Preparing metadata (setup.py) ... done
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24934 sha256=aaaf2b62732882403733e518919a0efd1e2ec6fd711df5624ae7310d23b1122f
  Stored in directory: /root/.cache/pip/wheels/1e/19/43/8a442dc83660ca25e163e1bd1f89919284ab0d0c1475475148
Successfully built rouge_score


When CUDAOutofMemoery errors occurred, running the following codes can eliminate some CUDAOutofMemory errors.
But some of them still cannot be fixed. Maybe a GPU computing platform with bigger memory is really needed.

In [ ]:
import torch
torch.cuda.empty_cache()
import gc
gc.collect()   ## garbage collection

30

 Code Assignment: ***Develop a question rephrasing model for the Disfl QA benchmark dataset***

 Code Author: **Dingjun Chen**

 Code Date: **From July 25 to July 30, 2025**

 Code Test Platform: ***Google Colab with its GPUs***

 **Introduction:** The following codes are using the pre-trained BART (Bidrecitional & AutoRegressive Transformer) to implement a question rephrasing model specifically tailored for the Disfl QA benchmark dataset. BART is designed to combine the strengths of BERT’s bidirectional encoding and GPT’s autoregressive generation. Thus, a fine-tuned BART variant is adept at tasks like summarization, translation, Question-Answering(QA) and sentiment analysis, etc. I am here to **use the fine-tuned BART model to extract the fluent questions from the disfluent questions, especially from those word-permutated disfluent questions**. In particular, I am using Hugging Face Transformers Library becaue it provides a high-level API for using pre-trained models, fine-tuning them on custom datasets, and deploying them in production environments.

In [ ]:
from copy import deepcopy
from transformers import BartTokenizer, BartForConditionalGeneration, TrainingArguments
from transformers import EvalPrediction, Trainer, TrainerCallback
import random, json, shutil,torch, os, evaluate
import numpy as np

Data augmentation is an optional technology to avoid overfitting in machine learning and it is the process of creating new samples by applying random transformations to the dataset.The following function is used to augment and **double training/validation datasets via the word permutation (shuffling words within a sentence)**.

In [ ]:
def DataAugmentation(src, label):
  size = len(src)
  for i in range(size):
    listword = src[i].split()
    random.shuffle(listword)
    sentence_new = ' '.join(listword)
    src.append(sentence_new)
    label.append(label[i])

Tokenization is the process of converting a sequence of text into tokens. The following  definitions of the class and function  are used to tokenize data. To create a PyTorch dataset, I define a class that inherits from the torch.utils.data.Dataset class. Both __getitem__ and __len__ methods are required to be implemented for creating custom datasets becasue the PyTorch Dataset class is a Python abstract base class.

In [ ]:
class BARTdataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        # super().__init__()
        self.encodings = encodings
        self.labels = labels
    ## To retrieve a specific sample and its corresponding label
    def __getitem__(self, index):
        # pass
        item = {key: torch.tensor(val[index]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels['input_ids'][index])
        return item
    ## To return the size of the dataset, meaning the total number of samples
    def __len__(self):
        # pass
        return len(self.labels['input_ids'])

def tokenize_dataset(texts, labels):
    tokenizer = BartTokenizer.from_pretrained('facebook/bart-large')
    encodings = tokenizer(texts,  padding=True)
    decodings = tokenizer(labels, padding=True)
    tokenized_dataset = BARTdataset(encodings, decodings)
    return tokenized_dataset

The following codes are used to customize the evaluation metrics and try some evaluation methods: accuracy, bleu or rouge, etc.,  otherwise the default evaluation would just have printed the loss.

***Note:***
The following code used for custom evaluation metrics are not successfully tested on Google Colab because OutOfMemoryError: "CUDA out of memory" occurred. These codes are developed based on my knowledge and understanding on custom evalation metrics. Once I have an opportunity to successfully test them on a GPU computing platform with bigger memory, I will update this repository in the future. All test results submitted for code assignment were generated in case of the default evaluation metric: LOSS.

In [ ]:
## Evaluation metrics with accuracy
def accuracy_score(eval_pred: EvalPrediction):
    metric = evaluate.load('accuracy')
    predictions, labels = eval_pred.predictions.argmax(axis=1), eval_pred.label_ids
    #logits, labels = eval_pred
    #predictions = np.argmax(logits, axis=1) ## Find indices of max values along rows (column-wise)
    return metric.compute(predictions=predictions, references=labels)

## Evaluation metrics with bleu
def bleu_score(eval_pred: EvalPrediction):
    metric = evaluate.load('bleu')
    predictions, labels = eval_pred.predictions.argmax(axis=1), eval_pred.label_ids
    #logits, labels = eval_pred
    #predictions = np.argmax(logits, axis=1) ## Find indices of max values along rows (column-wise)
    return metric.compute(predictions=predictions, references=labels)

## Evaluation metrics with rouge
def rouge_score(eval_pred: EvalPrediction):
    metric = evaluate.load('rouge')
    predictions, labels = eval_pred.predictions.argmax(axis=1), eval_pred.label_ids
    #logits, labels = eval_pred
    #predictions = np.argmax(logits, axis=1) ## Find indices of max values along rows (column-wise)
    return metric.compute(predictions=predictions, references=labels, use_aggregator=True)

## Combination of multiple evaluation metrics
def combination_compute_metrics1(eval_pred: EvalPrediction):
    ## Create dictionary to store Accuracy, Bleu and Rouge metrics
    metrics = {}  ## dictionary
    ## Compute Accuracy score
    metrics['Accuracy'] = accuracy_score(eval_pred)
    ## Compute Rouge score
    metrics['rouge1'], metrics['rougeL'] = rouge_score(eval_pred)
    ## Compute Bleu score
    metrics['bleu'] = bleu_score(eval_pred)
    return metrics

## Using a callback to trainer because Huggingface does not explicitly log the train accuracy
## Adding a custom callback which calls the evaluate() method with train_dataset at the end of every callback.
class CustomCallback(TrainerCallback):

    def __init__(self, trainer) -> None:
        super().__init__()
        self._trainer = trainer

    def on_epoch_end(self, args, state, control, **kwargs):
        if control.should_evaluate:
            control_copy = deepcopy(control) ## If not deep copy control, the trainer would not evaluate the evaluation dataset
            self._trainer.evaluate(eval_dataset=self._trainer.train_dataset, metric_key_prefix="train")
            return control_copy

## Combination of multiple evaluation metrics
def combination_compute_metrics2(eval_pred: EvalPrediction):
    metrics = ['accuracy', 'bleu']
    metric={}
    for i in metrics:
       metric[i] = evaluate.load(i)
    predictions, labels = eval_pred.predictions.argmax(axis=1), eval_pred.label_ids
    #logits, labels = eval_pred
    #predictions = np.argmax(logits, axis=1)
    metric_results={} ## Create dictionary to store Accuracy and Bleu metrics
    for i in metrics:
       metric_results[i]=metric[i].compute(predictions=predictions, references=labels)[i]
    return metric_results



Main program begins

To check if there is a CUDA GPU available

In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
if device == 'cuda':
  print('These codes are running on the device CUDA' )
else:
  print('These codes are running on the device CPU' )
print()

These codes are running on the device CUDA



To load the training dataset

In [ ]:
with open('train.json', 'r') as file:
  training_data = json.load(file)
training_src = []  ## list
training_label = [] ## list
for i in training_data:
  training_src.append(training_data[i]['disfluent'])
  training_label.append(training_data[i]['original'])

To load the validation dataset

In [ ]:
with open('dev.json', 'r') as file:
  validation_data = json.load(file)
validation_src = [] ## list
validation_label = [] ## list
for i in validation_data:
  validation_src.append(validation_data[i]['disfluent'])
  validation_label.append(validation_data[i]['original'])

To augment training/validation datasets.

If the original disfluent questions without data augmentation are only used for training, then you may comment out the following two statements.

In [ ]:
DataAugmentation(training_src, training_label)
DataAugmentation(validation_src, validation_label)

To tokenize datasets

In [ ]:
training_dataset = tokenize_dataset(training_src, training_label)
validation_dataset = tokenize_dataset(validation_src, validation_label)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

config.json: 0.00B [00:00, ?B/s]

To load a pre-trained model: BART

In [ ]:
model_name = 'facebook/bart-large'
model = BartForConditionalGeneration.from_pretrained(model_name).to(device)

pytorch_model.bin:   0%|          | 0.00/1.02G [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.02G [00:00<?, ?B/s]

To define training argumentsr

In [ ]:
training_args = TrainingArguments(
output_dir='./results',
num_train_epochs=5,
overwrite_output_dir=True,
#do_train=True,
#do_eval=True,
per_device_train_batch_size=16,
per_device_eval_batch_size=16,
learning_rate=5e-5,
#weight_decay=1e-2,
#lr_scheduler_type='cosine',
eval_strategy='epoch',
save_strategy='epoch',
#logging_strategy='epoch',
logging_dir='./logs',
logging_steps=20,
fp16=True,
load_best_model_at_end=True,

)

To create a Trainer instance

In [ ]:

trainer = Trainer(
  train_dataset=training_dataset,
  eval_dataset=validation_dataset,
  model=model,
  args=training_args,
  tokenizer=BartTokenizer.from_pretrained('facebook/bart-large'),

  ## To try different custom evaluaiton metrics
  ## otherwise using the default evaluation metric
  ## the default matric is 'loss'

  #compute_metrics=accuracy_score,
  #compute_metrics=bleu_score,
  #compute_metrics=rouge_score,
  #compute_metrics=combination_compute_metrics1,
  #compute_metrics=combination_compute_metrics2,

)

/tmp/ipython-input-15-1986452107.py:1: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


To fine-tune the BART model

In [ ]:
#trainer.add_callback(CustomCallback(trainer))
trainer.train()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize?ref=models
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: chendingjun68 (chendingjun68-self-employed) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Epoch,Training Loss,Validation Loss
1,0.271500,0.248141
2,0.124700,0.226405
3,0.080500,0.217437
4,0.045000,0.226649
5,0.034300,0.245225


/usr/local/lib/python3.11/dist-packages/transformers/modeling_utils.py:3854: UserWarning: Moving the following attributes in the config to the generation config: {'early_stopping': True, 'num_beams': 4, 'no_repeat_ngram_size': 3, 'forced_bos_token_id': 0}. You are seeing this warning because you've set generation parameters in the model config, as opposed to in the generation config.
  warnings.warn(
There were missing keys in the checkpoint model loaded: ['model.encoder.embed_tokens.weight', 'model.decoder.embed_tokens.weight', 'lm_head.weight'].


To delete non-empty direcotry

In [ ]:
directory = './model'
if os.path.isdir(directory):
  shutil.rmtree(directory)

To save the well-trained model into the specified directory

In [ ]:
model.save_pretrained(directory)